In [60]:
import boto3
import requests
from bs4 import BeautifulSoup

AWS_S3_NAME = "rearc-dataquest-s3bucket"
BLS_DATA = "https://download.bls.gov/pub/time.series/pr/"
headers = {
    "User-Agent": "rearc/1.0 (bhanu.tcs@gmail.com)"
}
response = requests.get(BLS_DATA, headers=headers)
print(response)
print(response.json)

## To initialize S3 resource and refer the specific bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket(AWS_S3_NAME)


<Response [200]>
<bound method Response.json of <Response [200]>>


In [61]:
print(bucket)

s3.Bucket(name='rearc-dataquest-s3bucket')


In [62]:
# Get bucket contents
bucket_objects = []
for obj in bucket.objects.all():
    bucket_objects.append(obj.key)
print(bucket_objects)

['pr.class', 'pr.contacts', 'pr.data.0.Current', 'pr.data.1.AllData', 'pr.duration', 'pr.footnote', 'pr.measure', 'pr.period', 'pr.seasonal', 'pr.sector', 'pr.series', 'pr.txt']


In [63]:
# Track files that don't exist on the website
deleted_list = bucket_objects.copy()
print(deleted_list)

['pr.class', 'pr.contacts', 'pr.data.0.Current', 'pr.data.1.AllData', 'pr.duration', 'pr.footnote', 'pr.measure', 'pr.period', 'pr.seasonal', 'pr.sector', 'pr.series', 'pr.txt']


In [64]:
# Request the data source and parse it
r = requests.get(BLS_DATA,headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
print(soup)

for link in soup.find_all("a"):
    # Download the current file
    file_name = link.get_text()
    if file_name == "[To Parent Directory]":
        continue
    file_dl = requests.get(BLS_DATA + file_name)
    # If the file doesn't exist in S3, upload it
    if file_name not in bucket_objects:
        bucket.put_object(Key=file_name, Body=file_dl.content)
    # If the file exists in S3
    elif file_name in bucket_objects:
        # Get the S3 file
        s3_response = bucket.Object(file_name).get()
        s3_file_content = s3_response['Body'].read()
        # If the S3 file is different from the website file, update the S3 file
        if file_dl.content != s3_file_content:
            bucket.put_object(Key=file_name, Body=file_dl.content)
        # Remove the file from the deleted list
        deleted_list.remove(file_name)
        print(deleted_list)
        
# Remove files from S3 that are no longer on the website
#for file in deleted_list:
#   if file != "population.json":
#        bucket.Object(file).delete()

<html><head><title>download.bls.gov - /pub/time.series/pr/</title></head><body><h1>download.bls.gov - /pub/time.series/pr/</h1><hr/>
<pre><a href="/pub/time.series/">[To Parent Directory]</a><br/><br/>  9/4/2025  8:30 AM          102 <a href="/pub/time.series/pr/pr.class">pr.class</a><br/> 9/13/2022  4:52 PM          562 <a href="/pub/time.series/pr/pr.contacts">pr.contacts</a><br/>  9/4/2025  8:30 AM      1564284 <a href="/pub/time.series/pr/pr.data.0.Current">pr.data.0.Current</a><br/>  9/4/2025  8:30 AM      3187878 <a href="/pub/time.series/pr/pr.data.1.AllData">pr.data.1.AllData</a><br/>  9/4/2025  8:30 AM          176 <a href="/pub/time.series/pr/pr.duration">pr.duration</a><br/>  9/4/2025  8:30 AM           40 <a href="/pub/time.series/pr/pr.footnote">pr.footnote</a><br/>  9/4/2025  8:30 AM          745 <a href="/pub/time.series/pr/pr.measure">pr.measure</a><br/>  1/7/1994  3:53 PM          146 <a href="/pub/time.series/pr/pr.period">pr.period</a><br/>11/18/2011  4:05 PM        

In [46]:
# Request the data source and parse it
import boto3
import requests
from bs4 import BeautifulSoup

AWS_S3_NAME = "rearc-dataquest-s3bucket"
DATA_SOURCE = "https://download.bls.gov/pub/time.series/pr/"
headers = {
    "User-Agent": "rearc/1.0 (bhanu.tcs@gmail.com)"
}
response = requests.get(DATA_SOURCE, headers=headers)

r = requests.get(DATA_SOURCE,headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
print(soup)

for link in soup.find_all("a"):
    # Download the current file
    file_name = link.get_text()
    if file_name == "[To Parent Directory]":
        continue
    file_dl = requests.get(DATA_SOURCE + file_name)
    # If the file doesn't exist in S3, upload it
    if file_name not in bucket_objects:
        bucket.put_object(Key=file_name, Body=file_dl.content)
    # If the file exists in S3
    elif file_name in bucket_objects:
        # Get the S3 file
        s3_response = bucket.Object(file_name).get()
        s3_file_content = s3_response['Body'].read()
        # If the S3 file is different from the website file, update the S3 file
        if file_dl.content != s3_file_content:
            bucket.put_object(Key=file_name, Body=file_dl.content)
        # Remove the file from the deleted list
        deleted_list.remove(file_name)
        print(deleted_list)

<html><head><title>download.bls.gov - /pub/time.series/pr/</title></head><body><h1>download.bls.gov - /pub/time.series/pr/</h1><hr/>
<pre><a href="/pub/time.series/">[To Parent Directory]</a><br/><br/>  9/4/2025  8:30 AM          102 <a href="/pub/time.series/pr/pr.class">pr.class</a><br/> 9/13/2022  4:52 PM          562 <a href="/pub/time.series/pr/pr.contacts">pr.contacts</a><br/>  9/4/2025  8:30 AM      1564284 <a href="/pub/time.series/pr/pr.data.0.Current">pr.data.0.Current</a><br/>  9/4/2025  8:30 AM      3187878 <a href="/pub/time.series/pr/pr.data.1.AllData">pr.data.1.AllData</a><br/>  9/4/2025  8:30 AM          176 <a href="/pub/time.series/pr/pr.duration">pr.duration</a><br/>  9/4/2025  8:30 AM           40 <a href="/pub/time.series/pr/pr.footnote">pr.footnote</a><br/>  9/4/2025  8:30 AM          745 <a href="/pub/time.series/pr/pr.measure">pr.measure</a><br/>  1/7/1994  3:53 PM          146 <a href="/pub/time.series/pr/pr.period">pr.period</a><br/>11/18/2011  4:05 PM        

ValueError: list.remove(x): x not in list